 # Event Log Preparation for Decision Mining: Triage Room Process

## Initial Setup

In [114]:
import pandas as pd
import numpy as np 


In [115]:
df_event_log = pd.read_csv('synthetic_triage_eventlog.csv')
print(df_event_log.head(10))

     Case ID Activity                    Activity Label       time:timestamp  \
0  202501011        A                Initial Assessment  2025-01-01T10:26:00   
1  202501011        B                     Immediate CPR  2025-01-01T10:44:00   
2  202501011        C      Stabilization and Monitoring  2025-01-01T11:11:00   
3  202501011        E   Further Diagnostics and Imaging  2025-01-01T12:07:00   
4  202501011        G  Treatment and Monitor Conditions  2025-01-01T12:19:00   
5  202501011        H               Transfer to the ICU  2025-01-01T13:12:00   
6  202501012        A                Initial Assessment  2025-01-01T08:49:00   
7  202501012        B                     Immediate CPR  2025-01-01T09:47:00   
8  202501012        C      Stabilization and Monitoring  2025-01-01T10:17:00   
9  202501012        E   Further Diagnostics and Imaging  2025-01-01T10:51:00   

   Heart_Rate  Oxygen_Saturation Blood_Pressure  Age Consciousness_Level  \
0          63               87.7         12

In [116]:
print(df_event_log.columns)

Index(['Case ID', 'Activity', 'Activity Label', 'time:timestamp', 'Heart_Rate',
       'Oxygen_Saturation', 'Blood_Pressure', 'Age', 'Consciousness_Level',
       'PHI', 'Imaging_Results'],
      dtype='object')


## Decision Point Labeling (DP1–DP4)

In [117]:
# Step 1: Convert timestamp column to datetime format and sort events by case and time
df_event_log['time:timestamp'] = pd.to_datetime(df_event_log['time:timestamp'])
df_event_log = df_event_log.sort_values(by=['Case ID', 'time:timestamp'])

# Step 2: Define a function to get the first activity that follows a given target activity
def get_next_after(activity_list, target):
    for i in range(len(activity_list) - 1):
        if activity_list[i] == target:
            return activity_list[i + 1]
    return None

# Step 3: Initialize dictionaries to store DP labels
dp1_labels = {}            # First activity after A
dp2_label_dict = {}        # Label determined based on C/D pattern before E
dp3_labels = {}            # First activity after E
dp4_labels = {}            # First activity after G

# Step 4: Process each case in the event log
grouped = df_event_log.groupby('Case ID')

for case_id, group in grouped:
    activities = list(group['Activity'].values)
    indices = group.index.tolist()

    # --- DP1: First activity after A ---
    dp1_labels[case_id] = get_next_after(activities, 'A')

    # --- DP3: First activity after E ---
    dp3_labels[case_id] = get_next_after(activities, 'E')

    # --- DP4: First activity after G ---
    dp4_labels[case_id] = get_next_after(activities, 'G')

    # --- DP2: Labeling logic before E ---
    try:
        e_index = activities.index('E')
        activities_before_e = activities[:e_index]
        indices_before_e = indices[:e_index]
    except ValueError:
        # Skip case if there is no activity E (Exception)
        continue

    # Count how many times C and D occurred before E
    c_count = activities_before_e.count('C')
    d_count = activities_before_e.count('D')
    total_cd = c_count + d_count

    # Initialize a label list for the whole trace
    case_labels = [''] * len(activities)

    if total_cd == 1:
        # Case 1: Only one C or D before E → all rows get label E
        case_labels = ['E'] * len(activities)
    else:
        # Case 2: Multiple occurrences → label based on next activity
        last_cd_index = -1
        for i in range(len(activities_before_e)):
            act = activities_before_e[i]
            if act == 'C':
                if i + 1 < len(activities_before_e):
                    next_act = activities_before_e[i + 1]
                    if next_act == 'B':
                        case_labels[i] = 'B'
                    elif next_act == 'C':
                        case_labels[i] = 'C'
                    elif next_act == 'D':
                        case_labels[i] = 'D'
                last_cd_index = i
            elif act == 'D':
                case_labels[i] = 'E'
                last_cd_index = i

        # After the last C or D, add label E to the next activity
        if last_cd_index >= 0 and last_cd_index + 1 < len(activities):
            case_labels[last_cd_index + 1] = 'E'

    # Map the generated DP2 labels to their original row indices
    for idx, label in zip(indices, case_labels):
        dp2_label_dict[idx] = label

# Step 5: Add all decision point labels back into the dataframe
df_event_log['DP1_label'] = df_event_log['Case ID'].map(dp1_labels)
df_event_log['DP2_label'] = df_event_log.index.map(dp2_label_dict)
df_event_log['DP3_label'] = df_event_log['Case ID'].map(dp3_labels)
df_event_log['DP4_label'] = df_event_log['Case ID'].map(dp4_labels)

# Step 6: Forward and Backward filling of DP2_label within each case
    # a. Replace empty strings with NaN in the 'DP2_label' column
df_event_log['DP2_label'] = df_event_log['DP2_label'].replace('', np.nan)

    # b. Forward-fill missing values in 'DP2_label' within each 'Case ID'
df_event_log['DP2_label'] = df_event_log.groupby('Case ID')['DP2_label'].ffill()

    # c. Backward-fill remaining NaNs (at the beginning of each case), aligned with the main DataFrame index
df_event_log['DP2_label'] = (
    df_event_log.groupby('Case ID')['DP2_label']
    .apply(lambda group: group.bfill())
    .reset_index(level=0, drop=True)
)

# Display the first 10 rows
#print(df_event_log.head(10))

# Preview updated dataframe
df_event_log[[ 'Heart_Rate', 'Oxygen_Saturation', 'Blood_Pressure', 'Age', 'Consciousness_Level', 'PHI', 'Imaging_Results',
              'DP1_label', 'DP2_label', 'DP3_label', 'DP4_label' ]].head(20)

,Heart_Rate,Oxygen_Saturation,Blood_Pressure,Age,Consciousness_Level,PHI,Imaging_Results,DP1_label,DP2_label,DP3_label,DP4_label
0,63,87.7,128/52,51,Drowsy,Critical,NaN,B,E,G,H
1,63,87.7,128/52,51,Drowsy,Critical,NaN,B,E,G,H
2,63,87.7,128/52,51,Drowsy,Critical,NaN,B,E,G,H
3,63,87.7,128/52,51,Drowsy,Critical,No Critical Injury Found,B,E,G,H
4,63,87.7,128/52,51,Drowsy,Critical,No Critical Injury Found,B,E,G,H
5,63,87.7,128/52,51,Drowsy,Critical,No Critical Injury Found,B,E,G,H
6,133,89.9,103/51,47,Drowsy,Critical,NaN,B,E,F,None
7,133,89.9,103/51,47,Drowsy,Critical,NaN,B,E,F,None
8,133,89.9,103/51,47,Drowsy,Critical,NaN,B,E,F,None
9,133,89.9,103/51,47,Drowsy,Critical,Confirmed Critical Injury,B,E,F,None


## Transfer Categorical Event Attributes
Transfer categorical event attributes to numeric values format for decision tree

In [118]:
# --- Split Blood_Pressure ---
df_event_log[['bp_sys', 'bp_dia']] = df_event_log['Blood_Pressure'].str.split('/', expand=True).astype('Int64')

# --- Map Consciousness_Level ---
consciousness_map = {
    'Unconscious': 0,
    'Drowsy': 1,
    'Lethargic': 2,
    'Alert': 3
}
df_event_log['cons_num'] = df_event_log['Consciousness_Level'].map(consciousness_map).astype('Int64')

# --- Map PHI ---
phi_map = {
    'Life-Threatening': 0,
    'Critical': 1,
    'Deteriorating': 2,
    'Stable': 3
}
df_event_log['PHI_num'] = df_event_log['PHI'].map(phi_map).astype('Int64')

# --- Map Imaging_Results (handle empty values as NaN first) ---
imaging_map = {
    'Confirmed Critical Injury': 1,
    'No Critical Injury Found': 0
}
df_event_log['Img_res_num'] = df_event_log['Imaging_Results'].map(imaging_map).astype('Int64')

# Preview updated dataframe
df_event_log[['Blood_Pressure', 'bp_sys', 'bp_dia', 
              'Consciousness_Level', 'cons_num',
              'PHI', 'PHI_num',
              'Imaging_Results', 'Img_res_num']].head(20)


,Blood_Pressure,bp_sys,bp_dia,Consciousness_Level,cons_num,PHI,PHI_num,Imaging_Results,Img_res_num
0,128/52,128,52,Drowsy,1,Critical,1,NaN,<NA>
1,128/52,128,52,Drowsy,1,Critical,1,NaN,<NA>
2,128/52,128,52,Drowsy,1,Critical,1,NaN,<NA>
3,128/52,128,52,Drowsy,1,Critical,1,No Critical Injury Found,0
4,128/52,128,52,Drowsy,1,Critical,1,No Critical Injury Found,0
5,128/52,128,52,Drowsy,1,Critical,1,No Critical Injury Found,0
6,103/51,103,51,Drowsy,1,Critical,1,NaN,<NA>
7,103/51,103,51,Drowsy,1,Critical,1,NaN,<NA>
8,103/51,103,51,Drowsy,1,Critical,1,NaN,<NA>
9,103/51,103,51,Drowsy,1,Critical,1,Confirmed Critical Injury,1


## Transfer DPs' Labels
Transfer DPs' labels to numeric values format for decision tree

In [119]:
# --- DP1_label: B → 0, D → 1 ---
dp1_map = {'B': 0, 'D': 1}
df_event_log['DP1_label_num'] = df_event_log['DP1_label'].map(dp1_map).astype('Int64')

# --- DP2_label: B → 0, C → 1, D → 2, E → 3 ---
dp2_map = {'B': 0, 'C': 1, 'D': 2, 'E': 3}
df_event_log['DP2_label_num'] = df_event_log['DP2_label'].map(dp2_map).astype('Int64')

# --- DP3_label: F → 0, G → 1 ---
dp3_map = {'F': 0, 'G': 1}
df_event_log['DP3_label_num'] = df_event_log['DP3_label'].map(dp3_map).astype('Int64')

# --- DP4_label: H → 0, I → 1 ---
dp4_map = {'H': 0, 'I': 1}
df_event_log['DP4_label_num'] = df_event_log['DP4_label'].map(dp4_map).astype('Int64')

df_event_log[[ 'DP1_label', 'DP1_label_num',
               'DP2_label', 'DP2_label_num',
               'DP3_label', 'DP3_label_num',
               'DP4_label', 'DP4_label_num' ]].head(20)


,DP1_label,DP1_label_num,DP2_label,DP2_label_num,DP3_label,DP3_label_num,DP4_label,DP4_label_num
0,B,0,E,3,G,1,H,0
1,B,0,E,3,G,1,H,0
2,B,0,E,3,G,1,H,0
3,B,0,E,3,G,1,H,0
4,B,0,E,3,G,1,H,0
5,B,0,E,3,G,1,H,0
6,B,0,E,3,F,0,None,<NA>
7,B,0,E,3,F,0,None,<NA>
8,B,0,E,3,F,0,None,<NA>
9,B,0,E,3,F,0,None,<NA>


In [120]:
# Save to CSV
df_event_log.to_csv("labeled_triage_eventlog.csv", index=False)

from IPython.display import FileLink
FileLink("labeled_triage_eventlog.csv")

C:\Users\eelha081\OneDrive - University of Ottawa\GitHub\sim2dm-triage\labeled_triage_eventlog.csv

## Creating labeled datasets for DT
Generating labeled datasets for training a decision tree, per every DP

In [121]:
print(df_event_log.columns)

Index(['Case ID', 'Activity', 'Activity Label', 'time:timestamp', 'Heart_Rate',
       'Oxygen_Saturation', 'Blood_Pressure', 'Age', 'Consciousness_Level',
       'PHI', 'Imaging_Results', 'DP1_label', 'DP2_label', 'DP3_label',
       'DP4_label', 'bp_sys', 'bp_dia', 'cons_num', 'PHI_num', 'Img_res_num',
       'DP1_label_num', 'DP2_label_num', 'DP3_label_num', 'DP4_label_num'],
      dtype='object')


### DP1 labeled event log

In [122]:
selected_columns_DP1 = ['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num', 'DP1_label_num']
df_selected_DP1 = df_event_log[selected_columns_DP1]

# For each Case ID, keep rows up to and including the first Activity == 'A' (remove events after DP1)
def remove_from_A(group):
    if 'A' in group['Activity'].values:
        idx = group[group['Activity'] == 'A'].index[0]
        return group.loc[:idx]  # include row with Activity == 'A'
    else:
        return group

# Apply group-wise without DeprecationWarning
groups = []
for case_id, group in df_selected_DP1.groupby('Case ID', sort=False):
    result = remove_from_A(group.copy())
    groups.append(result)

df_selected_DP1 = pd.concat(groups).reset_index(drop=True)

# Save to CSV
df_selected_DP1.to_csv("DP1_labeled_triage_eventlog.csv", index=False)

# from IPython.display import FileLink
# FileLink("DP1_labeled_triage_eventlog.csv")

df_selected_DP1[ ['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num',
               'DP1_label_num']].head(10)


,Case ID,Activity,Heart_Rate,Oxygen_Saturation,bp_sys,bp_dia,Age,cons_num,PHI_num,DP1_label_num
0,202501011,A,63,87.7,128,52,51,1,1,0
1,202501012,A,133,89.9,103,51,47,1,1,0
2,202501013,A,100,83.7,83,46,75,0,0,0
3,202501014,A,136,86.1,81,50,30,1,1,0
4,202501015,A,108,88.3,104,69,18,1,1,0
5,202501016,A,102,79.7,115,69,26,0,0,0
6,202501017,A,121,76.6,103,49,44,0,0,0
7,202501018,A,111,90.1,117,55,56,2,2,1
8,202501019,A,112,82.0,83,54,41,0,0,0
9,2025010110,A,97,87.0,83,68,46,1,1,0


### DP2 labeled event log

In [123]:
selected_columns_DP2 = ['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num', 'DP2_label_num']
df_selected_DP2 = df_event_log[selected_columns_DP2]

# For each Case ID, remove the row where Activity == 'E' and all following rows (remove events after DP2)
def remove_from_E(group):
    if 'E' in group['Activity'].values:
        idx = group[group['Activity'] == 'E'].index[0]
        return group.loc[:idx - 1]  # exclude the row where Activity == 'E'
    else:
        return group

# Apply the function group by group (avoiding DeprecationWarning)
groups = []
for case_id, group in df_selected_DP2.groupby('Case ID', sort=False):
    result = remove_from_E(group.copy())
    groups.append(result)

df_selected_DP2 = pd.concat(groups).reset_index(drop=True)

# Save to CSV
df_selected_DP2.to_csv("DP2_labeled_triage_eventlog.csv", index=False)

# from IPython.display import FileLink
# FileLink("DP2_labeled_triage_eventlog.csv")

df_selected_DP2[ ['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num',
               'DP2_label_num']].head(10)

,Case ID,Activity,Heart_Rate,Oxygen_Saturation,bp_sys,bp_dia,Age,cons_num,PHI_num,DP2_label_num
0,202501011,A,63,87.7,128,52,51,1,1,3
1,202501011,B,63,87.7,128,52,51,1,1,3
2,202501011,C,63,87.7,128,52,51,1,1,3
3,202501012,A,133,89.9,103,51,47,1,1,3
4,202501012,B,133,89.9,103,51,47,1,1,3
5,202501012,C,133,89.9,103,51,47,1,1,3
6,202501013,A,100,83.7,83,46,75,0,0,0
7,202501013,B,100,83.7,83,46,75,0,0,0
8,202501013,C,100,83.7,83,46,75,0,0,0
9,202501013,B,100,83.7,83,46,75,0,0,0


### DP3 labeled event log

In [124]:
# Fill missing Img_res_num values within each Case ID group using backward fill on the original DataFrame
df_event_log['Img_res_num'] = df_event_log.groupby('Case ID')['Img_res_num'].transform(lambda x: x.bfill())

selected_columns_DP3 = ['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num', 'Img_res_num', 'DP3_label_num']
df_selected_DP3 = df_event_log[selected_columns_DP3]

# For each Case ID, remove rows after the first occurrence of Activity == 'E' (remove events after DP3)
def truncate_after_E(group):
    if 'E' in group['Activity'].values:
        idx = group[group['Activity'] == 'E'].index[0]
        return group.loc[:idx]  # include the row where Activity == 'E'
    else:
        return group

# Apply truncation without triggering DeprecationWarning
groups = []
for case_id, group in df_selected_DP3.groupby('Case ID', sort=False):
    result = truncate_after_E(group.copy())
    groups.append(result)

df_selected_DP3 = pd.concat(groups).reset_index(drop=True)

# Save to CSV
df_selected_DP3.to_csv("DP3_labeled_triage_eventlog.csv", index=False)

# from IPython.display import FileLink
# FileLink("DP3_labeled_triage_eventlog.csv")

df_selected_DP3[ ['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num', 'Img_res_num',
               'DP3_label_num']].head(10)

,Case ID,Activity,Heart_Rate,Oxygen_Saturation,bp_sys,bp_dia,Age,cons_num,PHI_num,Img_res_num,DP3_label_num
0,202501011,A,63,87.7,128,52,51,1,1,0,1
1,202501011,B,63,87.7,128,52,51,1,1,0,1
2,202501011,C,63,87.7,128,52,51,1,1,0,1
3,202501011,E,63,87.7,128,52,51,1,1,0,1
4,202501012,A,133,89.9,103,51,47,1,1,1,0
5,202501012,B,133,89.9,103,51,47,1,1,1,0
6,202501012,C,133,89.9,103,51,47,1,1,1,0
7,202501012,E,133,89.9,103,51,47,1,1,1,0
8,202501013,A,100,83.7,83,46,75,0,0,0,1
9,202501013,B,100,83.7,83,46,75,0,0,0,1


### DP4 labeled event log

In [125]:
selected_columns_DP4 = ['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num', 'Img_res_num', 'DP4_label_num']
df_selected_DP4 = df_event_log[selected_columns_DP4]

# Step 1: Remove rows where DP4_label_num is missing (DP4 is not reached)
df_selected_DP4 = df_selected_DP4.dropna(subset=['DP4_label_num'])

# Step 2: Define a function that truncates after Activity == 'G' (remove events after DP4)
def truncate_after_G(group):
    if 'G' in group['Activity'].values:
        idx = group[group['Activity'] == 'G'].index[0]
        return group.loc[:idx]
    else:
        return group

# Step 3: Remove grouping column before apply, then merge after
groups = []
for case_id, group in df_selected_DP4.groupby('Case ID', sort=False):
    result = truncate_after_G(group.copy())
    groups.append(result)

# Combine all processed groups
df_selected_DP4 = pd.concat(groups).reset_index(drop=True)

# Save to CSV
df_selected_DP4.to_csv("DP4_labeled_triage_eventlog.csv", index=False)

# from IPython.display import FileLink
# FileLink("DP4_labeled_triage_eventlog.csv")

# Show first 10 rows
df_selected_DP4[['Case ID', 'Activity', 'Heart_Rate', 'Oxygen_Saturation', 'bp_sys', 'bp_dia', 'Age', 'cons_num', 'PHI_num', 'Img_res_num',
                 'DP4_label_num']].head(10)


,Case ID,Activity,Heart_Rate,Oxygen_Saturation,bp_sys,bp_dia,Age,cons_num,PHI_num,Img_res_num,DP4_label_num
0,202501011,A,63,87.7,128,52,51,1,1,0,0
1,202501011,B,63,87.7,128,52,51,1,1,0,0
2,202501011,C,63,87.7,128,52,51,1,1,0,0
3,202501011,E,63,87.7,128,52,51,1,1,0,0
4,202501011,G,63,87.7,128,52,51,1,1,0,0
5,202501013,A,100,83.7,83,46,75,0,0,0,0
6,202501013,B,100,83.7,83,46,75,0,0,0,0
7,202501013,C,100,83.7,83,46,75,0,0,0,0
8,202501013,B,100,83.7,83,46,75,0,0,0,0
9,202501013,C,100,83.7,83,46,75,0,0,0,0
